In [1]:
pip install networkx matplotlib flask requests ecdsa

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install ipywidgets pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
# --- CÔNG CỤ CLIENT ---
def create_wallet():
    priv = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    pub = priv.get_verifying_key()
    return binascii.hexlify(priv.to_string()).decode(), binascii.hexlify(pub.to_string()).decode()

def sign_transaction(priv_key_hex, pub_key_hex, receiver, amount):
    tx = {"sender": pub_key_hex, "receiver": receiver, "amount": amount}
    sk = ecdsa.SigningKey.from_string(binascii.unhexlify(priv_key_hex), curve=ecdsa.SECP256k1)
    sig = sk.sign(json.dumps(tx, sort_keys=True).encode())
    tx["signature"] = binascii.hexlify(sig).decode()
    return tx

# --- KHU VỰC DÙNG THỬ (Sửa thông tin ở đây rồi bấm Play để lấy JSON) ---

# 1. Tạo ví mới (Nếu chưa có thì chạy dòng này, có rồi thì comment lại)
my_priv, my_pub = create_wallet()
print(f"🔑 Private Key: {my_priv}")
print(f"🌍 Public Key:  {my_pub}")

# 2. Tạo giao dịch giả lập (Copy kết quả JSON bỏ vào Thunder Client)
receiver_address = "Vi_Nguoi_Khac"
amount_to_send = 15.5

signed_tx = sign_transaction(my_priv, my_pub, receiver_address, amount_to_send)
print("\n" + "="*20 + " COPY JSON DƯỚI ĐÂY VÀO THUNDER CLIENT " + "="*20)
print(json.dumps(signed_tx, indent=4))

🔑 Private Key: 8e611426b470360aceb55f2a6b134a3edabce85d1197f1781b0c7d419cbb4ccc
🌍 Public Key:  d1ee326e0ad5879233ee2359744eccf3dd29e8aeb2bbd9d835076aed20ce49ca4a725501be16e428b8ca770e176f0f81e8535ff56b9b3ef15b37b82f2bad73c3

==================== COPY JSON DƯỚI ĐÂY VÀO THUNDER CLIENT ====================
{
    "sender": "d1ee326e0ad5879233ee2359744eccf3dd29e8aeb2bbd9d835076aed20ce49ca4a725501be16e428b8ca770e176f0f81e8535ff56b9b3ef15b37b82f2bad73c3",
    "receiver": "Vi_Nguoi_Khac",
    "amount": 15.5,
    "signature": "c91747bf158638a80c2dd3bd632d4ef1ad50d1c21974dffccf899799efa1d1ed6936969fe5c0136a4e70d3e171523771b9d3866a60a4435ec49737aea13a37ff"
}


In [54]:
# --- FLASK SERVER ---
app = Flask(__name__)

@app.route('/chain', methods=['GET'])
def get_chain():
    chain_data = [b.__dict__ for b in blockchain_node.chain]
    return jsonify({"length": len(chain_data), "chain": chain_data}), 200

@app.route('/mine', methods=['POST'])
def mine():
    miner = request.get_json().get('miner')
    if not miner: return jsonify({"error": "Thiếu ví thợ đào"}), 400
    idx = blockchain_node.mine_pending_transactions(miner)
    return jsonify({"message": "Đào thành công", "block_index": idx}), 200

@app.route('/add_transaction', methods=['POST'])
def add_tx():
    data = request.get_json()
    try:
        result = blockchain_node.add_transaction(data['sender'], data['receiver'], float(data['amount']), data.get('signature'))
        if result: return jsonify({"message": "Giao dịch OK"}), 201
        return jsonify({"error": "Chữ ký sai"}), 406
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    print("🚀 Server đang chạy... (Bấm Stop trên thanh công cụ nếu muốn dừng)")
    app.run(port=5000)

🚀 Server đang chạy... (Bấm Stop trên thanh công cụ nếu muốn dừng)
 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


 * Running on http://127.0.0.1:5000

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Press CTRL+C to quit


127.0.0.1 - - [12/Nov/2025 16:53:41] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:53:42] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:53:42] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:53:42] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:53:42] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:53:43] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:53:43] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2025 16:54:26] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [12/Nov/2025 16:54:26] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [12/Nov/2025 16:54:44] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [12/Nov/2025 16:54:44] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [12/Nov/2025 16:55:51] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [12/Nov/2025 16:55:51] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [12/Nov/2025 16:55:51] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - -

In [ ]:
import hashlib
import json
import time
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ==========================================
# 1. LOGIC BLOCKCHAIN (Giữ nguyên)
# ==========================================
class Block:
    def __init__(self, index, transactions, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.timestamp = time.time()
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "transactions": self.transactions,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "nonce": self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        target = "0" * difficulty
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
        return self.hash

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 3
        self.pending_transactions = []
        self.mining_reward = 50

    def create_genesis_block(self):
        return Block(0, [], "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, sender, receiver, amount):
        self.pending_transactions.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })

    def mine_pending_transactions(self, miner_address):
        new_block = Block(len(self.chain), self.pending_transactions, self.get_latest_block().hash)
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)
        self.pending_transactions = [
            {"sender": "Network", "receiver": miner_address, "amount": self.mining_reward}
        ]

# Khởi tạo Blockchain
bc = Blockchain()

# ==========================================
# 2. GIAO DIỆN ĐIỀU KHIỂN (IPYWIDGETS)
# ==========================================

# --- Output Area (Nơi hiển thị kết quả) ---
out_log = widgets.Output()  # Log thông báo (Đào xong, gửi tiền...)
out_chain = widgets.Output() # Hiển thị bảng Blockchain

def render_chain():
    with out_chain:
        clear_output()
        if len(bc.chain) > 0:
            chain_data = []
            for block in bc.chain:
                chain_data.append({
                    "Index": block.index,
                    "Time": time.ctime(block.timestamp),
                    "Transactions": len(block.transactions),
                    "Hash": block.hash,
                    "Nonce": block.nonce
                })
            df = pd.DataFrame(chain_data)
            display(HTML("<h3>📜 Sổ cái Blockchain hiện tại:</h3>"))
            display(df)
        
        # Hiển thị hàng chờ
        display(HTML("<h4>⏳ Giao dịch đang chờ (Mempool):</h4>"))
        if bc.pending_transactions:
            display(bc.pending_transactions)
        else:
            print("Hàng chờ trống.")

# --- Các Nút & Ô nhập liệu ---

# 1. Khu vực Gửi tiền
txt_sender = widgets.Text(description="Người gửi:", value="Alice")
txt_receiver = widgets.Text(description="Người nhận:", value="Bob")
num_amount = widgets.FloatText(description="Số tiền:", value=10.0)
btn_send = widgets.Button(description="💸 Gửi Tiền", button_style='info')

def on_send_click(b):
    bc.add_transaction(txt_sender.value, txt_receiver.value, num_amount.value)
    with out_log:
        print(f"✅ Đã thêm giao dịch: {txt_sender.value} -> {txt_receiver.value} ({num_amount.value})")
    render_chain()

btn_send.on_click(on_send_click)

# 2. Khu vực Đào Coin
txt_miner = widgets.Text(description="Ví thợ đào:", value="Vi_Cua_Tui")
btn_mine = widgets.Button(description="⛏️ Đào Block", button_style='warning')

def on_mine_click(b):
    with out_log:
        print("⛏️ Đang đào... xin chờ chút...")
        bc.mine_pending_transactions(txt_miner.value)
        print(f"🎉 Đào thành công Block #{bc.get_latest_block().index}!")
    render_chain()

btn_mine.on_click(on_mine_click)

# --- Sắp xếp giao diện ---
ui_transaction = widgets.VBox([widgets.HTML("<b>Giao dịch mới:</b>"), txt_sender, txt_receiver, num_amount, btn_send])
ui_mining = widgets.VBox([widgets.HTML("<b>Khu vực Đào (Mining):</b>"), txt_miner, btn_mine])

ui_main = widgets.HBox([ui_transaction, ui_mining]) # Xếp 2 cột
ui_final = widgets.VBox([widgets.HTML("<h2>⛓️ BLOCKCHAIN DEMO (JUPYTER UI)</h2>"), ui_main, widgets.HTML("<hr>"), out_log, out_chain])

# Hiển thị lần đầu
render_chain()
display(ui_final)